In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Model
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Read Data
train = pd.read_excel(r"/content/drive/MyDrive/深度學習於醫學影像/train data/data.xlsx")
test = pd.read_excel(r"/content/drive/MyDrive/深度學習於醫學影像/test data/test.xlsx")

In [4]:
test = test.fillna(0)

In [5]:
# Dummy Lable Dataframe
count = list(range(0,800))
df_label = pd.DataFrame(columns=['主動脈硬化(鈣化)','動脈彎曲','肺野異常','肺紋增加','脊椎病變','肺尖肋膜增厚','心臟肥大'],index=count)
df_label = df_label.fillna(0)
# Dummy Lable
# '主動脈硬化(鈣化)','動脈彎曲','肺野異常','肺紋增加','脊椎病變','肺間肋膜增厚','心臟肥大'
for i in range(800):
    #print(i,train['label'][i])
    row_label = train['label'][i]
    if row_label == 'normal':
        continue
    if '主動脈硬化(鈣化)' in row_label:
        df_label['主動脈硬化(鈣化)'][i] = 1
    if '肺尖肋膜增厚' in row_label:
        df_label['肺尖肋膜增厚'][i] = 1
    if '動脈彎曲' in row_label:
        df_label['動脈彎曲'][i] = 1
    if '肺野異常' in row_label:
        df_label['肺野異常'][i] = 1
    if '肺紋增加' in row_label:
        df_label['肺紋增加'][i] = 1
    if '脊椎病變' in row_label:
        df_label['脊椎病變'][i] = 1
    if '心臟肥大' in row_label:
        df_label['心臟肥大'][i] = 1

In [6]:
df_label

,主動脈硬化(鈣化),動脈彎曲,肺野異常,肺紋增加,脊椎病變,肺尖肋膜增厚,心臟肥大
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
795,0,1,0,0,1,0,0
796,0,1,0,0,1,0,0
797,0,1,0,0,1,0,0
798,0,1,0,0,1,0,0


In [ ]:
# dicom to jpg
import cv2 as cv
import numpy as np
import pydicom as PDCM

def Dicom_to_Image(Path):
    DCM_Img = PDCM.read_file(Path)

    rows = DCM_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)
    cols = DCM_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)

    Instance_Number = int(DCM_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)

    Window_Center = int(DCM_Img.get(0x00281050).value) #Get window center from tag (0028, 1050)
    Window_Width = int(DCM_Img.get(0x00281051).value) #Get window width from tag (0028, 1051)

    Window_Max = int(Window_Center + Window_Width / 2)
    Window_Min = int(Window_Center - Window_Width / 2)

    if (DCM_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)

    if (DCM_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(DCM_Img.get(0x00281053).value)

    New_Img = np.zeros((rows, cols), np.uint8)
    Pixels = DCM_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): #if intensity is greater than max window
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): #if intensity is less than min window
                New_Img[i][j] = 0
            else:
                New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) #Normalize the intensities

    return New_Img, Instance_Number

def main():
    # Input Path
    input_path = '/content/drive/MyDrive/深度學習於醫學影像/train data/'
    for i in range(800):
      print(i,"start")
      Input_Image = input_path + train['category'][i] + '/' + train['Filename'][i]
      #Input_Image = '/content/drive/MyDrive/深度學習於醫學影像/train data/normal/220.dcm'

      Output_Image, Instance_Number = Dicom_to_Image(Input_Image)

      cv.imwrite('/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/'+train['Filename'][i]+'.jpg', Output_Image)
      print(i,"finished")

if __name__ == "__main__":
    main()


'\nimport cv2 as cv\nimport numpy as np\nimport pydicom as PDCM\n\ndef Dicom_to_Image(Path):\n    DCM_Img = PDCM.read_file(Path)\n\n    rows = DCM_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)\n    cols = DCM_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)\n\n    Instance_Number = int(DCM_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)\n\n    Window_Center = int(DCM_Img.get(0x00281050).value) #Get window center from tag (0028, 1050)\n    Window_Width = int(DCM_Img.get(0x00281051).value) #Get window width from tag (0028, 1051)\n\n    Window_Max = int(Window_Center + Window_Width / 2)\n    Window_Min = int(Window_Center - Window_Width / 2)\n\n    if (DCM_Img.get(0x00281052) is None):\n        Rescale_Intercept = 0\n    else:\n        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)\n\n    if (DCM_Img.get(0x00281053) is None):\n        Rescale_Slope = 1\n    else:\n        Rescale_Slope = int(DCM_Img.get(0x

In [7]:
# Read images and merge them into the training data X.
from tensorflow.keras.applications.vgg16 import preprocess_input
X = np.array([])
path = '/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/'
for i in range(800):
  input_path = path + train['Filename'][i] + '.jpg'
  # Load image and Resize into (224, 224) 
  img = image.load_img(input_path, target_size=(224, 224))
  # Show image
  #imgplot = plt.imshow(img)
  #plt.show()
  # Add one dimension and Become to (1, 224, 224, 3), which the last dimension is color.
  img2 = image.img_to_array(img)
  img2 = np.expand_dims(img2, axis=0)
  #print(img2.shape)
  if len(X.shape) == 1:
      X = img2
  else:
      # Combine the pixels of all images
      X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)
    
np.save('x_ray', X)

In [8]:
# Set VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False)

# Connect My Layer
x = base_model.output
#x = layers.Flatten()(x)
x = layers.GlobalAveragePooling2D()(x)
#x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
#x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(7, activation='sigmoid')(x)

# Set New Model inputs/outputs
model = Model(inputs=base_model.input, outputs=x)

# Original layers of VGG16 are not retrained
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='binary_crossentropy')

58892288/58889256 [==============================] - 1s 0us/step


In [9]:
# Make dataframe into arraay
y = np.array(df_label)

In [10]:
X.shape, y.shape

((800, 224, 224, 3), (800, 7))

In [11]:
model.fit(X, y, epochs=10, validation_split=0.2, verbose=2)

Epoch 1/10
20/20 - 402s - loss: 0.2328 - val_loss: 0.8742
Epoch 2/10
20/20 - 399s - loss: 0.1433 - val_loss: 0.6817
Epoch 3/10
20/20 - 399s - loss: 0.1265 - val_loss: 0.6572
Epoch 4/10
20/20 - 397s - loss: 0.1178 - val_loss: 0.6902
Epoch 5/10
20/20 - 402s - loss: 0.1137 - val_loss: 0.7376
Epoch 6/10
20/20 - 397s - loss: 0.1066 - val_loss: 0.6914
Epoch 7/10
20/20 - 394s - loss: 0.0986 - val_loss: 0.7237
Epoch 8/10
20/20 - 394s - loss: 0.0932 - val_loss: 0.6826
Epoch 9/10
20/20 - 394s - loss: 0.0877 - val_loss: 0.8318
Epoch 10/10
20/20 - 393s - loss: 0.0853 - val_loss: 0.7675


In [12]:
test_path = '/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg'    
for n in range(100):
  print(n)
  output_path = test_path + test['Filename'][n] + '.jpg'
  img = image.load_img(output_path, target_size=(224, 224))
  #imgplot = plt.imshow(img)
  #plt.show()
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model_matrix = model.predict(x)
  print("n = ",n)
  print(model_matrix)
  for i in range(7):
    print("n = ",n)
    print("model_matrix = ",model_matrix)
    test['主動脈硬化(鈣化)'][n] = model_matrix[0][0]
    test['動脈彎曲'][n] = model_matrix[0][1]
    test['肺野異常'][n] = model_matrix[0][2]
    test['肺紋增加'][n] = model_matrix[0][3]
    test['脊椎病變'][n] = model_matrix[0][4]
    test['肺間肋膜增厚'][n] = model_matrix[0][5]
    test['心臟肥大'][n] = model_matrix[0][6]

0
n =  0
[[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
n =  0
model_matrix =  [[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
n =  0
model_matrix =  [[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
n =  0
model_matrix =  [[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
n =  0
model_matrix =  [[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
n =  0
model_matrix =  [[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
n =  0
model_matrix =  [[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
n =  0
model_matrix =  [[0.01483721 0.00269392 0.00948891 0.00419137 0.00208032 0.01599407
  0.01819846]]
1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to b

n =  1
[[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
n =  1
model_matrix =  [[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
n =  1
model_matrix =  [[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
n =  1
model_matrix =  [[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
n =  1
model_matrix =  [[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
n =  1
model_matrix =  [[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
n =  1
model_matrix =  [[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
n =  1
model_matrix =  [[0.02816063 0.00056633 0.00373137 0.00276512 0.00183451 0.01396188
  0.00710595]]
2
n =  2
[[0.17392895 0.00581375 0.02862421 0.01718828 0.00471696 0.07136601
  0.04828247]]
n =  2
model_matrix =  [[0.17392895 0.00581375 0.02862421 0.01718828 0.00471

In [13]:
# Save VGG16 result
test.to_csv("/content/drive/MyDrive/深度學習於醫學影像/VGG16_new.csv")  